# Importar as Bibliotecas

In [1]:
%pip install pandas matplotlib folium plotly dash openpyxl nbformat>=4.2.0 ipython

import pandas as pd
import matplotlib.pyplot as plt
import folium
import plotly.graph_objects as go

Note: you may need to restart the kernel to use updated packages.


# Elaboração de mapa para observar situação dos núcleos quanto à membresia

In [2]:
# Carregar a planilha fornecida
file_path = 'content/Planilha Dashboard.xlsx'
df = pd.read_excel(file_path, sheet_name='Situação dos Núcleos')

# Substituir vírgulas por pontos e converter para float
df['LATITUDE'] = df['LATITUDE'].astype(str).str.replace(',', '.').astype(float)
df['LONGITUDE'] = df['LONGITUDE'].astype(str).str.replace(',', '.').astype(float)

# Remover linhas com valores inválidos de latitude e longitude
df = df.dropna(subset=['LATITUDE', 'LONGITUDE'])

In [3]:
# Criar o mapa centrado em uma coordenada média
mapa = folium.Map(location=[-15, -47], zoom_start=4)

# Função para determinar a cor do marcador com base na situação
def definir_cor(situacao):
    if situacao == 'PAGO':
        return 'green'
    elif situacao == 'ATRASADO':
        return 'red'
    elif situacao == 'DIVERGÊNCIA':
        return 'orange'
    else:
        return 'blue'

# Adicionar marcadores ao mapa
for _, row in df.iterrows():
    folium.Marker(
        location=[row['LATITUDE'], row['LONGITUDE']],
        popup=row['Descrição núcleo'],
        icon=folium.Icon(color=definir_cor(row['Situação']))
    ).add_to(mapa)

# Criar uma legenda personalizada com HTML
legend_html = '''
<div style="position: fixed;
     bottom: 50px; left: 50px; width: 150px; height: 150px;
     background-color: white; border:2px solid grey; z-index:9999; font-size:14px;
     ">
     <b>Legenda</b><br>
     <i class="fa fa-map-marker fa-2x" style="color:green"></i> PAGO<br>
     <i class="fa fa-map-marker fa-2x" style="color:red"></i> ATRASADO<br>
     <i class="fa fa-map-marker fa-2x" style="color:orange"></i> DIVERGÊNCIA<br>
     <i class="fa fa-map-marker fa-2x" style="color:blue"></i> OUTROS
</div>
'''

# Adicionar a legenda ao mapa
mapa.get_root().html.add_child(folium.Element(legend_html))

# Salvar o mapa em um arquivo HTML
mapa.save('content/mapa_situacao_nucleos.html')

# Exibir o mapa
mapa

# Gráfico de linhas para as Entradas e Saídas ao longo do ano.

In [4]:
# Carregar a planilha fornecida
df = pd.read_excel(file_path, sheet_name='Entradas e saidas')

# Converter a coluna "Data" para datetime e "Pago_Recebido" para numérico
df['Data'] = pd.to_datetime(df['Data'], format='%d/%m/%y')
df['Pago_Recebido'] = pd.to_numeric(df['Pago_Recebido'], errors='coerce')

# Remover valores nulos na coluna Pago_Recebido
df = df.dropna(subset=['Pago_Recebido'])

# Criar colunas para identificar receitas e despesas
df['Receitas'] = df['Pago_Recebido'].apply(lambda x: x if x > 0 else 0)
df['Despesas'] = df['Pago_Recebido'].apply(lambda x: x if x < 0 else 0)

# Agrupar receitas e despesas por mês
df['Mes'] = df['Data'].dt.to_period('M')
entradas_mes = df.groupby('Mes')['Receitas'].sum()
saidas_mes = df.groupby('Mes')['Despesas'].sum()

# Somar receitas e despesas para obter o total por mês
soma_mes = entradas_mes + saidas_mes

# Converter a coluna "Pago_Recebido" para tipo numérico
df['Pago_Recebido'] = pd.to_numeric(df['Pago_Recebido'], errors='coerce')

# Remover valores nulos que possam ter sido criados durante a conversão
df = df.dropna(subset=['Pago_Recebido'])

# Análise básica dos dados
saldo_total = df['Pago_Recebido'].sum()
print(f"Saldo total: R$ {saldo_total:.2f}")

# Separar receitas e despesas
receitas = df[df['Pago_Recebido'] > 0]
despesas = df[df['Pago_Recebido'] < 0]

# Total de receitas e despesas
total_receitas = receitas['Pago_Recebido'].sum()
total_despesas = despesas['Pago_Recebido'].sum()

print(f"Total de receitas: R$ {total_receitas:.2f}")
print(f"Total de despesas: R$ {total_despesas:.2f}")

Saldo total: R$ -43855.82
Total de receitas: R$ 38942.46
Total de despesas: R$ -82798.28


In [5]:
fig = go.Figure()

# Adicionar linha para entradas
fig.add_trace(go.Scatter(
    x=entradas_mes.index.astype(str),
    y=entradas_mes,
    mode='lines+markers',
    name='Entradas',
    line=dict(color='skyblue', width=2),
    fill='tozeroy',  # Preencher a área abaixo da linha
    fillcolor='rgba(135, 206, 235, 0.3)'  # Azul claro com transparência
))

# Adicionar linha para saídas
fig.add_trace(go.Scatter(
    x=saidas_mes.index.astype(str),
    y=saidas_mes,
    mode='lines+markers',
    name='Saídas',
    line=dict(color='lightsalmon', width=2),
    fill='tozeroy',
    fillcolor='rgba(255, 160, 122, 0.3)'  # Salmão claro com transparência
))

# Adicionar linha para a soma
fig.add_trace(go.Scatter(
    x=soma_mes.index.astype(str),
    y=soma_mes,
    mode='lines+markers',
    name='Soma',
    line=dict(color='darkgray', width=2)
))

# Atualizar layout
fig.update_layout(
    title='Performance ao Longo do Ano',
    xaxis_title='Meses',
    yaxis_title='Valor (R$)',
    xaxis_tickangle=-45,
    xaxis=dict(
        tickvals=entradas_mes.index.astype(str),
        ticktext=entradas_mes.index.astype(str),
    ),
    yaxis=dict(showgrid=True, gridcolor='rgba(0,0,0,0.2)'),
    legend=dict(x=0.85, y=0.05),
    hovermode='x unified'
)

# Exibir o gráfico
fig.show()

In [6]:
# Salvar o gráfico Plotly em um arquivo HTML
fig.write_html('content/Performance_ao_longo_do_ano.html')

# Gráfico em barras para as Entradas

In [7]:
# Criar gráfico
fig = go.Figure()

# Adicionar barras para entradas
fig.add_trace(go.Bar(
    x=entradas_mes.index.astype(str),  # Usar meses como texto
    y=entradas_mes,
    name='Entradas',
    marker_color='skyblue',
    text=[f'R${val:.2f}' for val in entradas_mes],
    textposition='outside',
))

# Atualizar layout para o eixo x ter 12 divisões
fig.update_layout(
    title={
        'text': 'Saídas Mensais',
        'x': 0.5,  # Centraliza o título horizontalmente
        'xanchor': 'center',
        'yanchor': 'top'
    },
    margin=dict(l=50, r=50, t=50, b=50),  # Ajusta as margens para manter o gráfico centralizado
    xaxis_title='Meses',
    yaxis_title='Valor (R$)',
    barmode='group',
    xaxis_tickangle=-30,
    xaxis=dict(
        tickvals=entradas_mes.index.astype(str),  # Definir valores dos ticks
        ticktext=entradas_mes.index.astype(str),  # Definir texto dos ticks
    )
)

# Exibir o gráfico
fig.show()

In [8]:
# Salvar o gráfico Plotly em um arquivo HTML
fig.write_html('content/Entradas_ao_longo_do_ano.html')

# Gráfico em barras para as Saídas

In [9]:
# Criar gráfico
fig = go.Figure()

# Adicionar barras para saídas
fig.add_trace(go.Bar(
    x=saidas_mes.index.astype(str),
    y=saidas_mes,
    name='Saídas',
    marker_color='lightsalmon',
    text=[f'R${val:.2f}' for val in saidas_mes],
    textposition='outside',
))

# Atualizar layout para o eixo x ter 12 divisões
fig.update_layout(
    title={
        'text': 'Saídas Mensais',
        'x': 0.5,  # Centraliza o título horizontalmente
        'xanchor': 'center',
        'yanchor': 'top'
    },
    xaxis_title='Meses',
    yaxis_title='Valor (R$)',
    barmode='group',
    xaxis_tickangle=-30,
    xaxis=dict(
        tickvals=saidas_mes.index.astype(str),  # Definir valores dos ticks
        ticktext=saidas_mes.index.astype(str),  # Definir texto dos ticks
    ),
    margin=dict(l=50, r=50, t=50, b=50),  # Ajusta as margens para manter o gráfico centralizado
)

# Exibir o gráfico
fig.show()

In [10]:
# Salvar o gráfico Plotly em um arquivo HTML
fig.write_html('content/Saidas_ao_longo_do_ano.html')

# Gráfico Donut para as Receitas

In [11]:
# Carregar a planilha fornecida
df = pd.read_excel(file_path, sheet_name='Receitas')
print(df.columns)

Index(['Receitas/Origem', 'TOTAL'], dtype='object')


In [12]:
# Calcular a soma total das receitas
total_receitas = df['TOTAL'].astype(float).sum()

# Calcular a porcentagem de cada categoria
df['PORCENTAGEM'] = df['TOTAL'].astype(float) / total_receitas * 100

# Separar categorias com mais de 5% e somar as outras
df_maior_5 = df[df['PORCENTAGEM'] > 5]
df_menor_5 = df[df['PORCENTAGEM'] <= 5]

# Adicionar uma linha para a categoria "Outros" se houver categorias menores que 5%
if not df_menor_5.empty:
    total_outros = df_menor_5['TOTAL'].astype(float).sum()
    outros = pd.DataFrame({
        'Receitas/Origem': ['Outros'],
        'TOTAL': [total_outros],
        'PORCENTAGEM': [total_outros / total_receitas * 100]
    })
    df_maior_5 = pd.concat([df_maior_5, outros], ignore_index=True)

# Classificar por ordem crescente
df_maior_5 = df_maior_5.sort_values(by='TOTAL', ascending=True)
# Criar gráfico de setores interativo
fig = go.Figure(data=[go.Pie(
    labels=df_maior_5['Receitas/Origem'],
    values=df_maior_5['TOTAL'].astype(float),
    textinfo='percent',  # Exibe porcentagem
    hovertemplate='%{label}<br>Valor: R$%{value:,.2f}<extra></extra>',  # Personaliza o hover
    hole=0.4  # Define o "buraco" no meio do gráfico, transformando-o em um donut
)])

# Atualizar layout do gráfico
fig.update_layout(
    title={
        'text': 'Distribuição de Despesas por Categoria',
        'x': 0.5,  # Centraliza o título horizontalmente
        'xanchor': 'center',
        'yanchor': 'top'
    },
    showlegend=True,
    legend=dict(
        x=0.5,  # Centraliza a posição horizontal da legenda
        y=-0.01,   # Ajusta a posição vertical da legenda (fora da área do gráfico)
        xanchor='center',  # Define o ponto de ancoragem da legenda ao centro
        yanchor='top',  # Define o ponto de ancoragem da legenda no topo
    ),
    margin=dict(l=50, r=50, t=50, b=50),  # Ajusta as margens para manter o gráfico centralizado
)

# Exibir o gráfico
fig.show()

In [13]:
# Salvar o gráfico Plotly em um arquivo HTML
fig.write_html('content/Donut_receitas.html')

# Gráfico Donut para as Despesas

In [14]:
# Carregar a planilha fornecida
df = pd.read_excel(file_path, sheet_name='Despesas')
print(df.columns)

Index(['Despesas', 'TOTAL'], dtype='object')


In [15]:
# Calcular a soma total das receitas
total_receitas = df['TOTAL'].astype(float).sum()

# Calcular a porcentagem de cada categoria
df['PORCENTAGEM'] = df['TOTAL'].astype(float) / total_receitas * 100

# Separar categorias com mais de 5% e somar as outras
df_maior_5 = df[df['PORCENTAGEM'] > 5]
df_menor_5 = df[df['PORCENTAGEM'] <= 5]

# Adicionar uma linha para a categoria "Outros" se houver categorias menores que 5%
if not df_menor_5.empty:
    total_outros = df_menor_5['TOTAL'].astype(float).sum()
    outros = pd.DataFrame({
        'Despesas': ['Outros'],
        'TOTAL': [total_outros],
        'PORCENTAGEM': [total_outros / total_receitas * 100]
    })
    df_maior_5 = pd.concat([df_maior_5, outros], ignore_index=True)

# Classificar por ordem crescente
df_maior_5 = df_maior_5.sort_values(by='TOTAL', ascending=True)
# Criar gráfico de setores interativo
fig = go.Figure(data=[go.Pie(
    labels=df_maior_5['Despesas'],
    values=df_maior_5['TOTAL'].astype(float),
    textinfo='percent',  # Exibe porcentagem
    hovertemplate='%{label}<br>Valor: R$%{value:,.2f}<extra></extra>',  # Personaliza o hover
    hole=0.4  # Define o "buraco" no meio do gráfico, transformando-o em um donut
)])

# Atualizar layout do gráfico
fig.update_layout(
    title={
        'text': 'Distribuição de Despesas por Categoria',
        'x': 0.5,  # Centraliza o título horizontalmente
        'xanchor': 'center',
        'yanchor': 'top'
    },
    showlegend=True,
    legend=dict(
        x=0.5,  # Centraliza a posição horizontal da legenda
        y=-0.01,   # Ajusta a posição vertical da legenda (fora da área do gráfico)
        xanchor='center',  # Define o ponto de ancoragem da legenda ao centro
        yanchor='top',  # Define o ponto de ancoragem da legenda no topo
    ),
    margin=dict(l=50, r=50, t=50, b=50),  # Ajusta as margens para manter o gráfico centralizado
)

# Exibir o gráfico
fig.show()

In [16]:
# Salvar o gráfico Plotly em um arquivo HTML
fig.write_html('content/Donut_despesas.html')